<a href="https://colab.research.google.com/github/Aunabil4602/Classification-on-COVID-19-Xray-datasets-using-Deep-Learning/blob/master/COVID19EfficientNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
%tensorflow_version 1.x

TensorFlow 1.x selected.


In [0]:
!pip install -U git+https://github.com/qubvel/efficientnet

In [0]:
import efficientnet.keras as efn 

Using TensorFlow backend.


In [0]:
model = efn.EfficientNetB7(weights='imagenet')

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
268320768/268317848 [==============================] - 3s 0us/step


In [0]:
model.summary()

In [0]:
model1 = efn.EfficientNetB7(weights='imagenet',include_top=False,input_shape=[300,300,3])

258441216/258434480 [==============================] - 3s 0us/step


In [0]:
model1.summary()

In [0]:

for layer in model1.layers:
  layer.trainable=False

In [0]:
from keras.layers import Flatten,Dense,MaxPooling2D,BatchNormalization,Conv2D,Dropout
from keras.models import Model

In [0]:
X=Conv2D(1024,(3,3),strides=(1,1),activation='relu',padding='SAME')(model1.output)
X=Conv2D(1024,(3,3),strides=(1,1),activation='relu',padding='SAME')(X)
X=BatchNormalization(axis=-1)(X)
X=MaxPooling2D(2,2)(X)
X=Dropout(0.25)(X)
X=Flatten()(X)
X=Dense(512,activation='relu')(X)
pred=Dense(2,activation='softmax')(X)

modelTL=Model(inputs=model1.input,outputs=pred)
modelTL.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
modelTL.summary()


Model: "model_2"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 150, 150, 64) 1728        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 150, 150, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 150, 150, 64) 0           stem_bn[0][0]                    
___________________________________________________________________________________________

In [0]:
!pip install kaggle

In [0]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"amlan107","key":"590556bfeabe7dd9d3f8acc4253a829c"}'}

In [0]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json

In [0]:
!kaggle datasets download -d praveengovi/coronahack-chest-xraydataset

100% 1.19G/1.19G [00:09<00:00, 162MB/s]
100% 1.19G/1.19G [00:09<00:00, 136MB/s]


In [0]:
!unzip coronahack-chest-xraydataset.zip

In [0]:
directory='./Coronahack-Chest-XRay-Dataset/Coronahack-Chest-XRay-Dataset/'
fileName='Chest_xray_Corona_Metadata.csv'

In [0]:
import pandas as pd

In [0]:
dataDetails=pd.read_csv('./'+fileName)

In [0]:
print(len(dataDetails))

5910


In [0]:
import os

In [0]:
os.mkdir('./data')
os.mkdir('./data/train')
os.mkdir('./data/test')
os.mkdir('./data/train/normal')
os.mkdir('./data/train/pnemonia')
os.mkdir('./data/test/normal')
os.mkdir('./data/test/pnemonia')

In [0]:
from shutil import copyfile 

In [0]:
total=5910

In [0]:
total_train=0
total_test=0
for it in range(5910):
  if dataDetails['Dataset_type'][it]=='TRAIN':
      total_train +=1
      fname=dataDetails['X_ray_image_name'][it]
      if dataDetails['Label'][it]=='Pnemonia':
        copyfile(directory+'train/'+fname,'./data/train/pnemonia/'+fname)
      else :
        copyfile(directory+'train/'+fname,'./data/train/normal/'+fname)
  else :
    total_test +=1
    fname=dataDetails['X_ray_image_name'][it]
    if dataDetails['Label'][it]=='Pnemonia':
      copyfile(directory+'test/'+fname,'./data/test/pnemonia/'+fname)
    else :
      copyfile(directory+'test/'+fname,'./data/test/normal/'+fname)

print(total_test)
print(total_train)

624
5286


In [0]:
print(total_train+total_test)

5910


In [0]:
from os import listdir

In [0]:
fileList1=listdir('./data/train/pnemonia/')
fileList2=listdir('./data/test/pnemonia/')
fileList3=listdir('./data/test/normal/')
fileList4=listdir('./data/train/normal/')

In [0]:
print(len(fileList1)+len(fileList2)+len(fileList3)+len(fileList4))

5910


In [0]:
##### generators
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator 

In [0]:
dataGen=ImageDataGenerator(rescale=1./255,rotation_range=5.0,zoom_range=[0.95,1.05],width_shift_range=0.05,height_shift_range=0.05,fill_mode='constant',cval=255)

In [0]:
dataGenOrigin=ImageDataGenerator(rescale=1./255)

In [0]:
trainSet=dataGen.flow_from_directory('./data/train',target_size=(300,300),batch_size=32,class_mode='categorical',color_mode='rgb')
testSet=dataGen.flow_from_directory('./data/test',target_size=(300,300),batch_size=32,class_mode='categorical',color_mode='rgb')

Found 5286 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [0]:
testSetOrigin=dataGenOrigin.flow_from_directory('./data/test',target_size=(300,300),batch_size=32,class_mode='categorical',color_mode='rgb')

Found 624 images belonging to 2 classes.


In [0]:

from keras.callbacks import ReduceLROnPlateau
reduce_lr = ReduceLROnPlateau(monitor='acc', factor=0.5,patience=3, min_lr=0.00001,verbose=1)

In [0]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=1,verbose=1)


Epoch 1/1
166/166 [==============================] - 151s 909ms/step - loss: 1.3701 - acc: 0.7649 - val_loss: 1.4056 - val_acc: 0.6603


In [0]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=20,verbose=1,callbacks=[reduce_lr])

Epoch 1/20
166/166 [==============================] - 141s 847ms/step - loss: 0.3564 - acc: 0.8369 - val_loss: 1.9218 - val_acc: 0.6571
Epoch 2/20
166/166 [==============================] - 133s 803ms/step - loss: 0.2828 - acc: 0.8835 - val_loss: 0.7956 - val_acc: 0.6603
Epoch 3/20
166/166 [==============================] - 133s 802ms/step - loss: 0.2269 - acc: 0.9069 - val_loss: 1.5827 - val_acc: 0.6554
Epoch 4/20
166/166 [==============================] - 133s 803ms/step - loss: 0.2002 - acc: 0.9222 - val_loss: 0.4790 - val_acc: 0.7099
Epoch 5/20
166/166 [==============================] - 134s 808ms/step - loss: 0.1779 - acc: 0.9291 - val_loss: 1.1751 - val_acc: 0.6811
Epoch 6/20
166/166 [==============================] - 133s 804ms/step - loss: 0.1693 - acc: 0.9340 - val_loss: 1.4802 - val_acc: 0.6426
Epoch 7/20
166/166 [==============================] - 134s 806ms/step - loss: 0.1489 - acc: 0.9414 - val_loss: 0.5614 - val_acc: 0.6571
Epoch 8/20
166/166 [============================

In [0]:
r1=modelTL.fit_generator(trainSet,validation_data=testSet,epochs=10,verbose=1,callbacks=[reduce_lr])

Epoch 1/10
166/166 [==============================] - 141s 850ms/step - loss: 0.1103 - acc: 0.9565 - val_loss: 1.1989 - val_acc: 0.6346
Epoch 2/10
166/166 [==============================] - 134s 804ms/step - loss: 0.1004 - acc: 0.9620 - val_loss: 0.6733 - val_acc: 0.6442
Epoch 3/10
166/166 [==============================] - 134s 806ms/step - loss: 0.1065 - acc: 0.9569 - val_loss: 0.9199 - val_acc: 0.6554
Epoch 4/10
166/166 [==============================] - 134s 809ms/step - loss: 0.0957 - acc: 0.9629 - val_loss: 0.6844 - val_acc: 0.6410
Epoch 5/10
166/166 [==============================] - 133s 801ms/step - loss: 0.1069 - acc: 0.9588 - val_loss: 1.4484 - val_acc: 0.6378
Epoch 6/10
166/166 [==============================] - 134s 809ms/step - loss: 0.1000 - acc: 0.9635 - val_loss: 2.1287 - val_acc: 0.6250
Epoch 7/10
166/166 [==============================] - 134s 804ms/step - loss: 0.1060 - acc: 0.9612 - val_loss: 1.5357 - val_acc: 0.6394
Epoch 8/10
166/166 [============================

In [0]:
e1=modelTL.evaluate_generator(testSetOrigin,verbose=1)
print(e1)

20/20 [==============================] - 10s 484ms/step
[0.5362878441810608, 0.7291666865348816]


In [0]:
X=MaxPooling2D(2,2)(model1.output)
X=Dropout(0.25)(X)
X=Flatten()(X)
X=Dense(1024,activation='relu')(X)
pred=Dense(2,activation='softmax')(X)

modelTL1=Model(inputs=model1.input,outputs=pred)
modelTL1.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['acc'])
modelTL1.summary()

Model: "model_4"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_2 (InputLayer)            (None, 300, 300, 3)  0                                            
__________________________________________________________________________________________________
stem_conv (Conv2D)              (None, 150, 150, 64) 1728        input_2[0][0]                    
__________________________________________________________________________________________________
stem_bn (BatchNormalization)    (None, 150, 150, 64) 256         stem_conv[0][0]                  
__________________________________________________________________________________________________
stem_activation (Activation)    (None, 150, 150, 64) 0           stem_bn[0][0]                    
____________________________________________________________________________________________

In [0]:
r2=modelTL1.fit_generator(trainSet,validation_data=testSet,epochs=10,verbose=1,callbacks=[reduce_lr])

Epoch 1/10
166/166 [==============================] - 145s 874ms/step - loss: 2.0414 - acc: 0.8617 - val_loss: 0.8004 - val_acc: 0.6458
Epoch 2/10
166/166 [==============================] - 132s 797ms/step - loss: 0.2180 - acc: 0.9232 - val_loss: 1.0991 - val_acc: 0.6587
Epoch 3/10
166/166 [==============================] - 133s 804ms/step - loss: 0.2087 - acc: 0.9349 - val_loss: 1.2488 - val_acc: 0.6490
Epoch 4/10
166/166 [==============================] - 134s 805ms/step - loss: 0.1642 - acc: 0.9444 - val_loss: 1.0503 - val_acc: 0.6538
Epoch 5/10
166/166 [==============================] - 133s 802ms/step - loss: 0.1407 - acc: 0.9465 - val_loss: 2.5737 - val_acc: 0.6250
Epoch 6/10
166/166 [==============================] - 132s 797ms/step - loss: 0.1546 - acc: 0.9434 - val_loss: 1.0118 - val_acc: 0.6859
Epoch 7/10
166/166 [==============================] - 134s 806ms/step - loss: 0.1348 - acc: 0.9480 - val_loss: 1.3342 - val_acc: 0.6410
Epoch 8/10
166/166 [============================

In [0]:
r2=modelTL1.fit_generator(trainSet,validation_data=testSet,epochs=20,verbose=1,callbacks=[reduce_lr])

Epoch 1/20
166/166 [==============================] - 139s 839ms/step - loss: 0.1459 - acc: 0.9487 - val_loss: 2.1903 - val_acc: 0.6234
Epoch 2/20
166/166 [==============================] - 134s 805ms/step - loss: 0.1287 - acc: 0.9535 - val_loss: 1.0262 - val_acc: 0.6346
Epoch 3/20
166/166 [==============================] - 134s 807ms/step - loss: 0.1876 - acc: 0.9482 - val_loss: 1.6730 - val_acc: 0.6282
Epoch 4/20
166/166 [==============================] - 134s 806ms/step - loss: 0.1809 - acc: 0.9493 - val_loss: 2.2457 - val_acc: 0.6426
Epoch 5/20
166/166 [==============================] - 133s 802ms/step - loss: 0.1926 - acc: 0.9502 - val_loss: 6.7990 - val_acc: 0.6250

Epoch 00005: ReduceLROnPlateau reducing learning rate to 0.0005000000237487257.
Epoch 6/20
166/166 [==============================] - 134s 806ms/step - loss: 0.1218 - acc: 0.9576 - val_loss: 1.5568 - val_acc: 0.6554
Epoch 7/20
166/166 [==============================] - 133s 800ms/step - loss: 0.1203 - acc: 0.9580 - va

In [0]:
e1=modelTL1.evaluate_generator(testSetOrigin,verbose=1)
print(e1)

18/20 [==========================>...] - ETA: 0s